<a href="https://colab.research.google.com/github/sai-teja-ponugoti/DeepBugs-for-Python/blob/master/Python/tokens_to_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import tarfile
# !wget http://files.srl.inf.ethz.ch/data/py150_files.tar.gz
# tf = tarfile.open("py150_files.tar.gz")
# tf.extractall()
# tf = tarfile.open("data.tar.gz")
# tf.extractall()

In [ ]:
# This program adds two numbers
import re
import os
from io import BytesIO
import tokenize
from tokenize import tok_name
import keyword

all_token_text_list = []
python_key_words = []
filepath = 'python100k_train.txt'
count = 0
with open(filepath) as fp:
    line = fp.readline()
    while line:
        # print(line)
        count += 1
        # if count>5:
            # break
            # print("finished ",count)
        if count%10000 == 0:
            print("finished ",count)

        string_of_tokens = []
        list_of_tokens = ''
        list_of_tokens = []

        # print(os.path.join(r"./", line.strip()))
        # break
        # with tokenize.open('/content/data/HenryHu/pybbs/digest.py') as f:.
        try:

            with tokenize.open(os.path.join(r"./", line.strip())) as f:
                # print(f.readline)
                # print(type(f.readline))
                # stri = re.sub(r"[\t]*", "", )
                # print("opened files")

                tokens = tokenize.generate_tokens(f.readline)
                # print("tokenized the file")
                for token in tokens:
                    # print(token.start)
                    if tok_name[token.type] == "NAME":
                        if token.string not in ['True', 'False', 'Null', 'None'] and token.string not in keyword.kwlist:
                            # list_of_tokens += "ID:" + token.string + "$%#~"
                            list_of_tokens.append("ID:" + token.string)
                            # string_of_tokens.append()
                        elif token.string in ['True', 'False', 'Null', 'None']:
                            # list_of_tokens += "LIT:" + token.string + "$%#~"
                            list_of_tokens.append("LIT:" + token.string)
                        elif token.string in keyword.kwlist:
                            # list_of_tokens += "STD:" + token.string + "$%#~"
                            list_of_tokens.append("STD:" + token.string)
                    elif tok_name[token.type] in ["NUMBER", "STRING"]:
                        # list_of_tokens += "LIT:" + token.string + "$%#~"
                        list_of_tokens.append("LIT:" + token.string)
                        
                    else:
                        # print(tok_name[token.type],end='')
                        # print(token.string,end='')
                        # print("done")
                        if token.string == '\n':
                            # list_of_tokens += r"STD:\n" + " "
                            list_of_tokens.append(r"STD:\n")
                        elif tok_name[token.type] == "INDENT":
                            # list_of_tokens += r"STD:\t" + "$%#~"
                            list_of_tokens.append(r"STD:\t")
                        elif tok_name[token.type] == "DEDENT":
                            # list_of_tokens += r"STD:DEDENT" + "$%#~"
                            list_of_tokens.append(r"STD:DEDENT")
                        elif tok_name[token.type] == "ENDMARKER":
                            # list_of_tokens += r"STD:ENDMARKER" + "$%#~"
                            list_of_tokens.append(r"STD:ENDMARKER")
                        else:
                            # list_of_tokens += "STD:" + token.string + "$%#~"
                            list_of_tokens.append("STD:" + token.string)
        except Exception as e:
            print("error occured at file ",count)
        all_token_text_list.append(list_of_tokens)
        line = fp.readline()

finished  10000
finished  20000
finished  30000
finished  40000
finished  50000
error occured at file  50521
error occured at file  55740
finished  60000
finished  70000
finished  80000
finished  90000
finished  100000


In [ ]:
len(all_token_text_list)

100000

In [ ]:

# all_token_text_list[0].index("LIT:True")
# all_token_text_list
# if 'STD:=' in all_token_text_list[0] :
    # print("Yes, 'at' found in List : " , all_token_text_list)

In [ ]:
from gensim.models import Word2Vec
import math
import time

import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

logging.root.level = logging.INFO

nb_tokens_in_context = 20
kept_tokens = 100000
embedding_size = 200 

model = Word2Vec(all_token_text_list,window=10, min_count=1,size=embedding_size, workers=400, iter=1)
time_stamp = math.floor(time.time() * 1000)
# model.save("embedding_model_" + str(time_stamp))

2020-08-17 03:45:36,357 : INFO : collecting all words and their counts
2020-08-17 03:45:36,358 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-17 03:45:38,205 : INFO : PROGRESS: at sentence #10000, processed 9480249 words, keeping 499959 word types
2020-08-17 03:45:40,123 : INFO : PROGRESS: at sentence #20000, processed 19164202 words, keeping 899515 word types
2020-08-17 03:45:41,986 : INFO : PROGRESS: at sentence #30000, processed 28669927 words, keeping 1253255 word types
2020-08-17 03:45:43,906 : INFO : PROGRESS: at sentence #40000, processed 38024563 words, keeping 1586247 word types
2020-08-17 03:45:45,886 : INFO : PROGRESS: at sentence #50000, processed 48132335 words, keeping 1918703 word types
2020-08-17 03:45:47,748 : INFO : PROGRESS: at sentence #60000, processed 57649591 words, keeping 2238247 word types
2020-08-17 03:45:49,697 : INFO : PROGRESS: at sentence #70000, processed 67562656 words, keeping 2544456 word types
2020-08-17 03:45:51,9

In [ ]:
import json
token_to_vector = dict()
for token in model.wv.vocab:
        if token.startswith("ID:") or token.startswith("LIT:"):
            vector = model[token].tolist()
            token_to_vector[token] = vector
# token_to_vector_file_name = "token_to_vector_" + str(time_stamp) + ".json"
with open("token_to_vector_new.json", "w") as file:
    json.dump(token_to_vector, file, sort_keys=True, indent=4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
# # model.wv
# model.wv.similarity('ID:num1', 'ID:num2')
# # model.wv.similarity('STD:\\n', 'STD:\\t')


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.26086137

In [ ]:
# model.running_training_los

0.0